## Text classification: Spam or Ham

In this example based on the classical dataset Spambase Dataset (https://archive.ics.uci.edu/ml/datasets/spambase) we will try to make our own spam filter using scikit-learn library. The dataset contains text corpora of  5.574 text messages with labels "spam" or "ham". 

### Data

Data are attached to the task description for your convinience

In [1]:
import pandas as pd
df = pd.read_csv('3_data.csv', encoding='latin-1')

We delete all other columns except for two of interest: text messages and labels:

In [2]:
df = df[['v1', 'v2']]
df = df.rename(columns = {'v1': 'label', 'v2': 'text'})
df.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


Delete duplicates:

In [3]:
df = df.drop_duplicates('text')

Change labels to binary:

In [4]:
df['label'] = df['label'].map({'ham': 0, 'spam': 1})

### Text pre-processing (Task)

We need to complete the function for text pre-processing, to pre-process the text the following way:
* convert text to lowercase;
* remove stop-words;
* remove punctuation marks;
* normalizes the text using Snowball stemmer.

We recommend to use the NLTK library, in order not to compile a list of stop-words and not to implement the stemming algorithm yourself. Click the link to find the examples of stemmers application (https://www.nltk.org/howto/stem.html).

In [5]:
# Function to convert 
def listToString(s):
   
    # initialize an empty string
    str1 = " "
   
    # return string 
    return (str1.join(s))

In [6]:
from nltk import stem
import nltk
from nltk.corpus import stopwords
import re
nltk.download('stopwords')
nltk.download('punkt')

stemmer = stem.SnowballStemmer('english')
stopwords = set(stopwords.words('english'))

def preprocess(text):
    text = re.sub(r'[^\w\s]', '', text)
    filtered_sentence = [stemmer.stem(w.lower()) for w in text.split(sep=" ") if not w.lower() in stopwords]
    filtered_sentence = listToString(filtered_sentence)
    
    # your code here
    return filtered_sentence

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ivan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ivan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Check that the function works correctly

In [7]:
assert preprocess("I'm gonna be home soon and i don't want to talk about this stuff anymore tonight, k? I've cried enough today.") == "im gonna home soon dont want talk stuff anymor tonight k ive cri enough today"
assert preprocess("Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...") == "go jurong point crazi avail bugi n great world la e buffet cine got amor wat"

Apply to the text:

In [8]:
df['text'] = df['text'].apply(preprocess)
df['text']

0       go jurong point crazi avail bugi n great world...
1                                   ok lar joke wif u oni
2       free entri 2 wkli comp win fa cup final tkts 2...
3                     u dun say earli hor u c alreadi say
4               nah dont think goe usf live around though
                              ...                        
5567    2nd time tri 2 contact u u å750 pound prize 2 ...
5568                             ì_ b go esplanad fr home
5569                             piti  mood soani suggest
5570    guy bitch act like id interest buy someth els ...
5571                                       rofl true name
Name: text, Length: 5169, dtype: object

### Split the data to the training and test set

In [9]:
y = df['label'].values

Now we need to split the data to test (test) and training (train) sets. Scikit-learn library contains ready to use tools to do it.

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['text'], y, test_size=0.25, random_state=63)
X_test

5468    urgent last weekend draw show å1000 cash spani...
2070    sexi singl wait text age follow gender wither ...
5332                      think steyn sure get one wicket
204                                    u call alter 11 ok
3394                                                  buy
                              ...                        
4140    beauti truth  express face could seen everyon ...
2358           ill talk other probabl come earli tomorrow
4848    either way work  ltgt  year old hope doesnt bo...
3144                                ill get tomorrow send
5408                                                  pub
Name: text, Length: 1293, dtype: object

### Classifier training

We came to the classifier training now.

First we extract features from the texts. It is strongly recommened to try several methods in order to check how each method influences the result (more information on defferent text representation methods you can find on the link https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction).

Then we train the classifier. We use SVM, but you can try different algorithms.

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# exctract features from the texts
vectorizer = TfidfVectorizer(decode_error='ignore')
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [12]:
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

#train SVM model

model = LinearSVC(random_state = 63, C = 1.5)
model.fit(X_train, y_train)
predictions = model.predict(X_test)

Selfcheck. If the function ```preprocess``` is complimented correctly, then you should get the following model evaluation results.

In [13]:
print(classification_report(y_test, predictions, digits=3))

              precision    recall  f1-score   support

           0      0.980     0.996     0.988      1133
           1      0.972     0.856     0.910       160

    accuracy                          0.979      1293
   macro avg      0.976     0.926     0.949      1293
weighted avg      0.979     0.979     0.979      1293



Let's predict results for the specified text

In [22]:
txt = "Machine factory for sale. Low price. 2 hectare property, 150,000 square feet production floor, 500 machine tools installed."
txt = preprocess(txt)
txt = vectorizer.transform([txt])

In [23]:
model.predict(txt)

array([0], dtype=int64)

The message is classified as spam.